In [7]:

from pathlib import Path
from ReChisel.testcase import Testcase

bm = 'verilog-eval'

if bm == 'verilog-eval':
    bmcase = Testcase(
        'Prob030_popcount255', 
        'benchmarks/VerilogEval_sample/Prob030_popcount255_spec.txt',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_ref.sv',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_tb.sv'
    )
    output_dir = Path('output/VerilogEval/')
    output_dir.mkdir(parents=True, exist_ok=True)

elif bm == 'autochip':
    bmcase = Testcase(
        'Vector5',
        'benchmarks/AutoChip_sample/Vector5_spec.txt',
        None, # No reference file provided
        'benchmarks/AutoChip_sample/Vector5_0_tb.v'
    )
    output_dir = Path('output/AutoChip/')
    output_dir.mkdir(parents=True, exist_ok=True)


print(bmcase.specification)

I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in  (255 bits)
 - output out (  8 bits)

A "population count" circuit counts the number of '1's in an input
vector. The module should implement a population count circuit for a
255-bit input vector.


In [8]:

from rechisel import initial_chisel_generation
from ReChisel.chisel_code import ChiselCode


target_path = output_dir / 'gen.scala'

if target_path.exists():
    chisel_code = ChiselCode(f"```scala\n\n{target_path.read_text()}\n\n```", 'TopModule')
else:
    chisel_code = initial_chisel_generation(bmcase, 'TopModule')
    target_path.write_text(chisel_code.decorated)

print(chisel_code.decorated)


package TopModule

import chisel3._
import chisel3.util._

import chisel3.stage.ChiselStage

class TopModule extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(255.W))
    val out = Output(UInt(8.W))
  })

  // Population count using the Chisel built-in PopCount method
  io.out := PopCount(io.in).asUInt(8.W)
}

object Main extends App {
    (new ChiselStage).emitVerilog(
      new TopModule,
      Array(
        "--target-dir", "generated",
        "--emission-options=disableMemRandomization,disableRegisterRandomization",
      )
    )
}



In [9]:

from ReChisel.verifier import Verifier, VerifierWorkingSpace, VerifyResult


verifier = Verifier(
    VerifierWorkingSpace(str(output_dir / 'chisel'), str(output_dir / 'iv')),
    bm
)

is_ok = (
    verifier.prepare(chisel_code, bmcase) and
    verifier.compile() and
    verifier.iv() and
    verifier.vvp() and
    verifier.functionality()
)
verify_result: VerifyResult = verifier.result

is_ok

False

In [10]:
import json

with open(output_dir / 'verify.json', 'w') as fp:
    json.dump(verify_result.__dict__, fp, indent=2)